In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [26]:
# We Separate the dataset
from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y)

(99913, 19)

In [8]:
function = ridge_regression
degree_vec = []
lambda_vec = []
for i in range(3):
    print(i)
    rmse_te, BestDeg, BestLambda = grid_search(y_list[i], tX_list[i], function, True)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    print(BestDeg)
    print(BestLambda)
    print(rmse_te)

0
1
1e-06
0.7174871793166815
1
3
1e-06
0.806441000152771
2
5
2.06913808111479e-05
0.7693986023405923
3
5
2.06913808111479e-05
0.7738287893438908


In [10]:
# degree_vec = [1, 3, 3, 4]
# lambda_vec = [0.00013738237958832623, 0.0001, 0.0001, 0.0001]

weights_list = []
loss = []
mat_tX_test_list = []
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)   #separate dateset

for i in range(4):
    print(i)
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], True, tX_test_list[i])
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss.append(l)
    mat_tX_test_list.append(mat_tX_test)
    

0
1
2
3


In [97]:
np.logspace(-4, 0, 30)


array([1.00000000e-04, 1.37382380e-04, 1.88739182e-04, 2.59294380e-04,
       3.56224789e-04, 4.89390092e-04, 6.72335754e-04, 9.23670857e-04,
       1.26896100e-03, 1.74332882e-03, 2.39502662e-03, 3.29034456e-03,
       4.52035366e-03, 6.21016942e-03, 8.53167852e-03, 1.17210230e-02,
       1.61026203e-02, 2.21221629e-02, 3.03919538e-02, 4.17531894e-02,
       5.73615251e-02, 7.88046282e-02, 1.08263673e-01, 1.48735211e-01,
       2.04335972e-01, 2.80721620e-01, 3.85662042e-01, 5.29831691e-01,
       7.27895384e-01, 1.00000000e+00])

## Do your thing crazy machine learning thing here :) ...

In [ ]:
# Linear regression using gradient descent

initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 3e-7

weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)

In [76]:
# Least squares regression using normal equation
weights, loss = least_squares(y, tX)
loss

0.3396868094770349

In [9]:
# Ridge regression using normal equations
lambda_ = 1e-3
weights, loss = ridge_regression(y, tX, lambda_)
loss

(30,)

In [59]:
# Ridge regression with data augmentation
lambda_ = 1e-3
mat_tX = build_poly(tX, 5)
weights, loss = ridge_regression(y, mat_tX, lambda_)

loss

0.2967432202164457

In [35]:
# Ridge regression with separated data
lambda_ = 1e-3
weights_list, loss_list = separated_train(tX_list, y_list, ridge_regression, lambda_)

In [76]:
# Ridge regression on separated datset with data augmentation
lambda_ = 1e-3
mat_tX_list = build_poly_separated(tX_list, 3)
# weights, loss = ridge_regression(y_list[0], mat_tX_list[0], lambda_)

weights_list, loss_list = separated_train(mat_tX_list, y_list, ridge_regression, lambda_)

## Generate predictions and save ouput in csv format for submission:

In [9]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [38]:
# Data augmentation
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
mat_tX_test = build_poly(tX_test, 3)
print(mat_tX_test.shape)
y_pred = predict_labels(weights, mat_tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

(568238, 240)


ValueError: shapes (568238,240) and (4,30) not aligned: 240 (dim 1) != 4 (dim 0)

In [41]:
# Separated dataset
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)
y_pred_list = separated_eval(weights_list, tX_test_list)

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

In [12]:
# Data augmentation and separated dataset
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)   #separate dateset

mat_tX_test_list = []

# for i in range(4):
#     print(i)
#     mat_tX_test = build_poly(tX_test_list[i], degree_vec[i])
#     mat_tX_test_list.append(mat_tX_test)

# mat_tX_test_list = build_poly_separated(tX_test_list, 3)            #data augmentation
y_pred_list = separated_eval(weights_list, mat_tX_test_list)        #prediction

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

0
1
2
3


In [12]:
OUTPUT_PATH = 'result.csv'
weights_list = []
loss = []
mat_tX_test_list = []
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)   #separate dateset

for i in range(3):
    print(i)
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], True, tX_test_list[i])
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss.append(l)
    mat_tX_test_list.append(mat_tX_test)

y_pred_list = separated_eval(weights_list, mat_tX_test_list) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

0
1
2
3
